In [121]:
import os
import pandas as pd
import numpy as np
# Show all columns
pd.set_option('display.max_columns', None)
import warnings

warnings.filterwarnings("ignore")

In [2]:
!nvidia-smi

zsh:1: command not found: nvidia-smi


## Download image data using provided download CLI
Below we download all metadata and resized images (500p) for a FungiTastic-Mini subset.

In [3]:
# !python ../../dataset/download.py --metadata --images --subset "m" --size "500" --save_path "./"

In [4]:
!ls ./FungiTastic/metadata/FungiTastic-Mini/

FungiTastic-Mini-ClosedSet-Test.csv FungiTastic-Mini-OpenSet-Test.csv
FungiTastic-Mini-ClosedSet-Val.csv  FungiTastic-Mini-OpenSet-Val.csv
FungiTastic-Mini-DNA-Test.csv       FungiTastic-Mini-Train.csv


In [5]:
!ls ./FungiTastic/FungiTastic-Mini

dna-test test     train    val


## Load and preprocess metadata

In [205]:
print(os.getcwd())

/Users/jeremycui/Documents/UCB_MIDS/DATASCI207/fungitastic-classification-datasci207-Fall-2025/baselines/closed_set


In [206]:
BASE_METADATA_PATH = f"{os.getcwd()}/FungiTastic/metadata/FungiTastic-Mini/"

train_df = pd.read_csv(f"{BASE_METADATA_PATH}FungiTastic-Mini-Train.csv")
val_df = pd.read_csv(f"{BASE_METADATA_PATH}FungiTastic-Mini-ClosedSet-Val.csv")
test_df = pd.read_csv(f"{BASE_METADATA_PATH}FungiTastic-Mini-ClosedSet-Test.csv")

### Setting image_path to the images.

In [207]:
BASE_IMAGE_PATH = "./FungiTastic/FungiTastic-Mini"
SIZE = "500p"

train_df["image_path"] = train_df.filename.apply(lambda filename: f"{BASE_IMAGE_PATH}/train/{SIZE}/{filename}")
val_df["image_path"] = val_df.filename.apply(lambda filename: f"{BASE_IMAGE_PATH}/val/{SIZE}/{filename}")

In [235]:
train_df

,eventDate,year,month,day,habitat,countryCode,scientificName,kingdom,phylum,class,order,family,genus,specificEpithet,hasCoordinate,species,iucnRedListCategory,substrate,latitude,longitude,coorUncert,observationID,region,district,filename,category_id,metaSubstrate,poisonous,elevation,landcover,biogeographicalRegion,image_path
0,2021-02-01,2021,2.0,1.0,Mixed woodland (with coniferous and deciduous ...,DK,Mycena tintinnabulum (Paulet) Quél.,Fungi,Basidiomycota,Agaricomycetes,Agaricales,Mycenaceae,Mycena,tintinnabulum,True,Mycena tintinnabulum,NE,dead wood (including bark),55.280440,11.761377,15.0,3032614317,Sjælland,Næstved,0-3032614317.JPG,119,wood,0,35.0,5.0,continental,./FungiTastic/FungiTastic-Mini/train/500p/0-30...
1,2021-02-01,2021,2.0,1.0,Mixed woodland (with coniferous and deciduous ...,DK,Mycena tintinnabulum (Paulet) Quél.,Fungi,Basidiomycota,Agaricomycetes,Agaricales,Mycenaceae,Mycena,tintinnabulum,True,Mycena tintinnabulum,NE,dead wood (including bark),55.280440,11.761377,15.0,3032614317,Sjælland,Næstved,1-3032614317.JPG,119,wood,0,35.0,5.0,continental,./FungiTastic/FungiTastic-Mini/train/500p/1-30...
2,2008-09-01,2008,9.0,1.0,Deciduous woodland,DK,Russula cyanoxantha (Schaeff.) Fr.,Fungi,Basidiomycota,Agaricomycetes,Russulales,Russulaceae,Russula,cyanoxantha,True,Russula cyanoxantha,NE,soil,56.249168,10.169204,75.0,3036761318,Midtjylland,Århus,0-3036761318.JPG,144,jord,0,6.0,10.0,continental,./FungiTastic/FungiTastic-Mini/train/500p/0-30...
3,2008-09-01,2008,9.0,1.0,Deciduous woodland,DK,Russula cyanoxantha (Schaeff.) Fr.,Fungi,Basidiomycota,Agaricomycetes,Russulales,Russulaceae,Russula,cyanoxantha,True,Russula cyanoxantha,NE,soil,56.249168,10.169204,75.0,3036761318,Midtjylland,Århus,1-3036761318.JPG,144,jord,0,6.0,10.0,continental,./FungiTastic/FungiTastic-Mini/train/500p/1-30...
4,2008-09-01,2008,9.0,1.0,Deciduous woodland,DK,Russula cyanoxantha (Schaeff.) Fr.,Fungi,Basidiomycota,Agaricomycetes,Russulales,Russulaceae,Russula,cyanoxantha,True,Russula cyanoxantha,NE,soil,56.249168,10.169204,75.0,3036761318,Midtjylland,Århus,2-3036761318.JPG,144,jord,0,6.0,10.0,continental,./FungiTastic/FungiTastic-Mini/train/500p/2-30...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46834,2021-12-19,2021,12.0,19.0,Mixed woodland (with coniferous and deciduous ...,DK,Clitocybe fragrans Sowerby,Fungi,Basidiomycota,Agaricomycetes,Agaricales,Tricholomataceae,Clitocybe,fragrans,True,Clitocybe fragrans,NE,soil,56.130876,10.205789,5.0,3422350324,Midtjylland,Århus,3-3422350324.JPG,50,jord,0,9.0,12.0,continental,./FungiTastic/FungiTastic-Mini/train/500p/3-34...
46835,2021-12-19,2021,12.0,19.0,Mixed woodland (with coniferous and deciduous ...,DK,Clitocybe fragrans Sowerby,Fungi,Basidiomycota,Agaricomycetes,Agaricales,Tricholomataceae,Clitocybe,fragrans,True,Clitocybe fragrans,NE,soil,56.130876,10.205789,5.0,3422350324,Midtjylland,Århus,4-3422350324.JPG,50,jord,0,9.0,12.0,continental,./FungiTastic/FungiTastic-Mini/train/500p/4-34...
46839,2021-09-27,2021,9.0,27.0,Deciduous woodland,DK,Russula ochroleuca Fr.,Fungi,Basidiomycota,Agaricomycetes,Russulales,Russulaceae,Russula,ochroleuca,True,Russula ochroleuca,NE,soil,55.954525,11.907830,6.0,4100099773,Hovedstaden,Halsnæs,0-4100099773.JPG,179,jord,0,0.0,17.0,continental,./FungiTastic/FungiTastic-Mini/train/500p/0-41...
46840,2021-09-27,2021,9.0,27.0,Deciduous woodland,DK,Russula ochroleuca Fr.,Fungi,Basidiomycota,Agaricomycetes,Russulales,Russulaceae,Russula,ochroleuca,True,Russula ochroleuca,NE,soil,55.954525,11.907830,6.0,4100099773,Hovedstaden,Halsnæs,1-4100099773.JPG,179,jord,0,0.0,17.0,continental,./FungiTastic/FungiTastic-Mini/train/500p/1-41...


In [211]:
train_df.dropna(inplace=True)
test_df.dropna(inplace=True)
val_df.dropna(inplace=True)

In [214]:
train_df.columns

Index(['eventDate', 'year', 'month', 'day', 'habitat', 'countryCode',
       'scientificName', 'kingdom', 'phylum', 'class', 'order', 'family',
       'genus', 'specificEpithet', 'hasCoordinate', 'species',
       'iucnRedListCategory', 'substrate', 'latitude', 'longitude',
       'coorUncert', 'observationID', 'region', 'district', 'filename',
       'category_id', 'metaSubstrate', 'poisonous', 'elevation', 'landcover',
       'biogeographicalRegion', 'image_path'],
      dtype='object')

### Preprocess Metadata

In [215]:
target_feat_list = ['species']
train_target = train_df[target_feat_list]
val_target = val_df[target_feat_list]

In [216]:
metadata_feat_list = [
    'habitat', 'countryCode',
    'scientificName', 'kingdom', 'phylum', 'class', 'order', 'family',
    'genus', 'specificEpithet', 'hasCoordinate',
    'iucnRedListCategory', 'substrate', 'latitude', 'longitude',
    'coorUncert', 'region', 'district',
    'metaSubstrate', 'poisonous', 'elevation', 'landcover',
    'biogeographicalRegion'
]
train_df_metadata = train_df[metadata_feat_list]
val_df_metadata = val_df[metadata_feat_list]
# test_df_metadata = test_df[metadata_feat_list]

In [175]:
train_df_metadata.describe()

,latitude,longitude,coorUncert,poisonous,elevation,landcover
count,44428.000000,44428.000000,44428.000000,44428.000000,44428.000000,44428.000000
mean,55.767944,11.064675,70.098857,0.128433,68.598626,11.568538
std,0.899763,1.600670,324.563724,0.334575,140.990427,5.498775
min,39.096246,-9.261015,1.000000,0.000000,0.000000,1.000000
25%,55.388833,9.829539,10.000000,0.000000,0.000000,8.000000
50%,55.784139,11.376017,25.000000,0.000000,13.000000,12.000000
75%,56.090752,12.291957,50.000000,0.000000,72.000000,17.000000
max,69.706248,29.473609,24069.000000,1.000000,2313.000000,17.000000


### Frequency Encoding

In [176]:
cat_metadata_feat_list = [
    'habitat', 'countryCode',
    'scientificName', 'kingdom', 'phylum', 'class', 'order', 'family',
    'genus', 'specificEpithet', 'hasCoordinate',
    'iucnRedListCategory', 'substrate', 'coorUncert', 'region', 'district', 
    'metaSubstrate', 'biogeographicalRegion'
]

cat_train_df_metadata = train_df_metadata[cat_metadata_feat_list]
cat_val_df_metadata = val_df_metadata[cat_metadata_feat_list]

In [177]:
train_cat_vc_dict = {}
val_cat_vc_dict = {}

for cat in list(cat_train_df_metadata.columns):
    
    train_cat_vc_dict[cat] = dict(cat_train_df_metadata[cat].value_counts())
    val_cat_vc_dict[cat] = dict(cat_val_df_metadata[cat].value_counts())

In [178]:
train_cat_vc_dict

{'habitat': {'Deciduous woodland': np.int64(10895),
  'Mixed woodland (with coniferous and deciduous trees)': np.int64(8136),
  'coniferous woodland/plantation': np.int64(5935),
  'Unmanaged deciduous woodland': np.int64(4750),
  'park/churchyard': np.int64(3414),
  'Unmanaged coniferous woodland': np.int64(1588),
  'natural grassland': np.int64(1515),
  'lawn': np.int64(1355),
  'roadside': np.int64(1130),
  'garden': np.int64(1025),
  'Thorny scrubland': np.int64(980),
  'Forest bog': np.int64(504),
  'Bog woodland': np.int64(486),
  'hedgerow': np.int64(394),
  'wooded meadow, grazing forest': np.int64(350),
  'dune': np.int64(326),
  'bog': np.int64(269),
  'heath': np.int64(262),
  'Willow scrubland': np.int64(219),
  'salt meadow': np.int64(208),
  'Acidic oak woodland': np.int64(148),
  'other habitat': np.int64(148),
  'meadow': np.int64(141),
  'gravel or clay pit': np.int64(92),
  'improved grassland': np.int64(69),
  'fallow field': np.int64(34),
  'ditch': np.int64(26),
  '

In [179]:
train_df_metadata_cat_enc = train_df_metadata.copy()

In [180]:
for col, mapping in train_cat_vc_dict.items():
    # Use .map with dict (fast, vectorized, memory-friendly)
    train_df_metadata_cat_enc[col] = train_df_metadata[col].map(mapping)

In [181]:
val_df_metadata

,habitat,countryCode,scientificName,kingdom,phylum,class,order,family,genus,specificEpithet,hasCoordinate,iucnRedListCategory,substrate,latitude,longitude,coorUncert,region,district,metaSubstrate,poisonous,elevation,landcover,biogeographicalRegion
0,dune,DK,Mycena chlorantha (Fr.) P.Kumm.,Fungi,Basidiomycota,Agaricomycetes,Agaricales,Mycenaceae,Mycena,chlorantha,True,NE,"dead stems of herbs, grass etc",55.549599,12.247782,25.0,Sjælland,Solrød,plant material,0,98.0,9.0,continental
1,coniferous woodland/plantation,DK,Clitocybe vibecina (Fr.) Quél.,Fungi,Basidiomycota,Agaricomycetes,Agaricales,Tricholomataceae,Clitocybe,vibecina,True,NE,soil,56.091144,9.329936,15.0,Midtjylland,Silkeborg,jord,0,122.0,1.0,atlantic
2,coniferous woodland/plantation,DK,Clitocybe vibecina (Fr.) Quél.,Fungi,Basidiomycota,Agaricomycetes,Agaricales,Tricholomataceae,Clitocybe,vibecina,True,NE,soil,56.091144,9.329936,15.0,Midtjylland,Silkeborg,jord,0,122.0,1.0,atlantic
3,roadside,DK,Mycena vitilis (Fr.) Quél.,Fungi,Basidiomycota,Agaricomycetes,Agaricales,Mycenaceae,Mycena,vitilis,True,NE,wood chips or mulch,55.435580,12.196362,2.0,Sjælland,Køge,wood,0,104.0,5.0,continental
4,roadside,DK,Mycena vitilis (Fr.) Quél.,Fungi,Basidiomycota,Agaricomycetes,Agaricales,Mycenaceae,Mycena,vitilis,True,NE,wood chips or mulch,55.435580,12.196362,2.0,Sjælland,Køge,wood,0,104.0,5.0,continental
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9407,Thorny scrubland,DK,Mycena juniperina Aronsen,Fungi,Basidiomycota,Agaricomycetes,Agaricales,Mycenaceae,Mycena,juniperina,True,NE,bark of living trees,56.644395,9.810166,5.0,Nordjylland,Mariagerfjord,wood,0,0.0,17.0,continental
9408,Thorny scrubland,DK,Mycena juniperina Aronsen,Fungi,Basidiomycota,Agaricomycetes,Agaricales,Mycenaceae,Mycena,juniperina,True,NE,bark of living trees,56.644395,9.810166,5.0,Nordjylland,Mariagerfjord,wood,0,0.0,17.0,continental
9409,Thorny scrubland,DK,Mycena juniperina Aronsen,Fungi,Basidiomycota,Agaricomycetes,Agaricales,Mycenaceae,Mycena,juniperina,True,NE,bark of living trees,56.644395,9.810166,5.0,Nordjylland,Mariagerfjord,wood,0,0.0,17.0,continental
9410,Mixed woodland (with coniferous and deciduous ...,DK,Amanita muscaria (L.) Lam.,Fungi,Basidiomycota,Agaricomycetes,Agaricales,Amanitaceae,Amanita,muscaria,True,NE,leaf or needle litter,55.474200,12.030310,5.0,Sjælland,Køge,plant material,1,74.0,12.0,continental


In [182]:
val_df_metadata_cat_enc = val_df_metadata.copy()

In [220]:
for col, mapping in val_cat_vc_dict.items():
    # Use .map with dict (fast, vectorized, memory-friendly)
    val_df_metadata_cat_enc[col] = val_df_metadata[col].map(mapping)

In [221]:
val_df_metadata_cat_enc.isna().sum().sum()

np.int64(0)

### Coordinates Parsing

In [222]:
train_df_metadata_cat_enc["lat_sin"] = np.sin(np.radians(train_df_metadata_cat_enc["latitude"]))
train_df_metadata_cat_enc["lat_cos"] = np.cos(np.radians(train_df_metadata_cat_enc["latitude"]))
train_df_metadata_cat_enc["lon_sin"] = np.sin(np.radians(train_df_metadata_cat_enc["longitude"]))
train_df_metadata_cat_enc["lon_cos"] = np.cos(np.radians(train_df_metadata_cat_enc["longitude"]))

In [223]:
val_df_metadata_cat_enc["lat_sin"] = np.sin(np.radians(val_df_metadata_cat_enc["latitude"]))
val_df_metadata_cat_enc["lat_cos"] = np.cos(np.radians(val_df_metadata_cat_enc["latitude"]))
val_df_metadata_cat_enc["lon_sin"] = np.sin(np.radians(val_df_metadata_cat_enc["longitude"]))
val_df_metadata_cat_enc["lon_cos"] = np.cos(np.radians(val_df_metadata_cat_enc["longitude"]))

### Target Label encoding

In [224]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
train_df_target_enc = le.fit_transform(train_target)
val_df_target_enc = le.transform(val_target)

In [225]:
from sklearn.preprocessing import StandardScaler

std_scaler = StandardScaler()
train_arr_metadata_cat_enc = std_scaler.fit_transform(train_df_metadata_cat_enc)
val_arr_metadata_cat_enc = std_scaler.transform(val_df_metadata_cat_enc)

In [226]:
train_df_metadata_cat_enc.columns

Index(['habitat', 'countryCode', 'scientificName', 'kingdom', 'phylum',
       'class', 'order', 'family', 'genus', 'specificEpithet', 'hasCoordinate',
       'iucnRedListCategory', 'substrate', 'latitude', 'longitude',
       'coorUncert', 'region', 'district', 'metaSubstrate', 'poisonous',
       'elevation', 'landcover', 'biogeographicalRegion', 'lat_sin', 'lat_cos',
       'lon_sin', 'lon_cos'],
      dtype='object')

### Preprocess Images

## Logistic Regression on Encoded Metadata

In [228]:
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier

clf_logreg_ovr = OneVsRestClassifier(LogisticRegression(max_iter=1000))
clf_logreg_ovr.fit(train_arr_metadata_cat_enc, train_df_target_enc)  # Y is binarized labels from MultiLabelBinarizer

,estimator,LogisticRegre...max_iter=1000)
,n_jobs,None
,verbose,0
,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None


In [231]:
# Predict on validation data
Y_pred_logreg_ovr = clf_logreg_ovr.predict(val_arr_metadata_cat_enc)

In [232]:
Y_pred_logreg_ovr

array([44, 44, 44, ..., 44, 23, 45], shape=(9391,))

In [233]:
val_df_target_enc

array([71, 60, 60, ..., 86, 32, 43], shape=(9391,))

In [234]:
from sklearn.metrics import accuracy_score, classification_report

print("Accuracy:", accuracy_score(val_df_target_enc, Y_pred_logreg_ovr))

Accuracy: 0.020338622084974976


### Update the column name to fit the FGVC expected naming convention and save the updated metadata

In [10]:
!mkdir metadata

train_df = train_df.rename(columns={"category_id":"class_id"})
val_df = val_df.rename(columns={"category_id":"class_id"})

train_df.to_csv("./metadata/FungiTastic-Mini-Train.csv", index=None)
val_df.to_csv("./metadata/FungiTastic-Mini-ClosedSet-Val.csv", index=None)

mkdir: metadata: File exists


## Train single model with a config file.

### Environment variables
Select which GPU to use and add your WANDB_ENTITY and WANDB_PROJECT you want to log into etc.

In [11]:
%env CUDA_DEVICES = 0
%env WANDB_ENTITY = zcu_cv
%env HFHUB_OWNER = BVRA
%env WANDB_PROJECT = FungiTastic
%env TRAIN_METADATA_PATH = ./metadata/FungiTastic-Mini-Train.csv
%env VAL_METADATA_PATH = ./metadata/FungiTastic-Mini-ClosedSet-Val.csv
%env TEST_METADATA_PATH = ./metadata/FungiTastic-Mini-ClosedSet-Test.csv

os.environ["CUDA_VISIBLE_DEVICES"] = os.environ["CUDA_DEVICES"]

env: CUDA_DEVICES=0
env: WANDB_ENTITY=zcu_cv
env: HFHUB_OWNER=BVRA
env: WANDB_PROJECT=FungiTastic
env: TRAIN_METADATA_PATH=./metadata/FungiTastic-Mini-Train.csv
env: VAL_METADATA_PATH=./metadata/FungiTastic-Mini-ClosedSet-Val.csv
env: TEST_METADATA_PATH=./metadata/FungiTastic-Mini-ClosedSet-Test.csv


In [14]:
!which python
!python -m pip install wandb

/Users/jeremycui/Documents/UCB_MIDS/DATASCI207/fungitastic-classification-datasci207-Fall-2025/.dataset_demo_venv/bin/python
  Using cached wandb-0.22.0-py3-none-macosx_12_0_arm64.whl.metadata (10 kB)
  Using cached click-8.3.0-py3-none-any.whl.metadata (2.6 kB)
  Using cached gitpython-3.1.45-py3-none-any.whl.metadata (13 kB)
  Using cached protobuf-6.32.1-cp39-abi3-macosx_10_9_universal2.whl.metadata (593 bytes)
  Using cached gitdb-4.0.12-py3-none-any.whl.metadata (1.2 kB)
  Using cached smmap-5.0.2-py3-none-any.whl.metadata (4.3 kB)
Using cached wandb-0.22.0-py3-none-macosx_12_0_arm64.whl (18.3 MB)
Using cached protobuf-6.32.1-cp39-abi3-macosx_10_9_universal2.whl (426 kB)
Using cached click-8.3.0-py3-none-any.whl (107 kB)
Using cached gitpython-3.1.45-py3-none-any.whl (208 kB)
Using cached gitdb-4.0.12-py3-none-any.whl (62 kB)
Using cached smmap-5.0.2-py3-none-any.whl (24 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7/7 [wandb]32m6/7 [wandb]uf]k]

[notice] A new release of pip i

In [16]:
!python train.py \
    --train-path "./metadata/FungiTastic-Mini-Train.csv" \
    --val-path "./metadata/FungiTastic-Mini-ClosedSet-Val.csv" \
    --test-path "./metadata/FungiTastic-Mini-ClosedSet-Test.csv" \
    --config-path ./configs/default_config.yaml \
    --cuda-devices $CUDA_DEVICES \
    --wandb-entity $WANDB_ENTITY \
    --wandb-project $WANDB_PROJECT \
    --hfhub-owner $HFHUB_OWNER

(script) INFO: Loading training config.
(script) DEBUG: Extra arguments passed to the script: {}
(script) INFO: Setting run name: resnet18.a1_in1k-CrossEntropyLoss-light
(script) INFO: Using experiment directory: ../runs/resnet18.a1_in1k-CrossEntropyLoss-light/exp2
(script) INFO: Using training configuration: {
    "augmentations": "light",
    "image_size": [
        224,
        224
    ],
    "dataset": "FungiTastic-Mini",
    "architecture": "resnet18.a1_in1k",
    "train": null,
    "loss": "CrossEntropyLoss",
    "optimizer": "SGD",
    "scheduler": "plateau",
    "epochs": 1,
    "learning_rate": 0.01,
    "batch_size": 64,
    "accumulation_steps": 1,
    "random_seed": 777,
    "workers": 8,
    "multigpu": false,
    "tags": [
        "FungiTastic-M-CloseSet",
        "224x224",
        "Convolutional"
    ],
    "root_path": "../",
    "run_name": "resnet18.a1_in1k-CrossEntropyLoss-light",
    "exp_name": "exp2",
    "exp_path": "../runs/resnet18.a1_in1k-CrossEntropyLoss-lig

## Running Sweep Training

### Environment variables
Select which GPU to use and add your WANDB_ENTITY and WANDB_PROJECT you want to log into etc.

In [ ]:
%env CUDA_DEVICES = "changethis"
%env WANDB_ENTITY = "changethis"
%env HFHUB_OWNER = "changethis"
%env WANDB_PROJECT = "changethis"
%env TRAIN_METADATA_PATH = ./metadata/FungiTastic-Mini-Train.csv
%env VAL_METADATA_PATH = ./metadata/FungiTastic-Mini-ClosedSet-Val.csv
%env TEST_METADATA_PATH = ./metadata/FungiTastic-Mini-ClosedSet-Test.csv

os.environ["CUDA_VISIBLE_DEVICES"] = os.environ["CUDA_DEVICES"]

In [ ]:
!wandb sweep ./sweep_configs/FungiTastic-M-CloseSet-224.yaml

In [ ]:
!wandb agent <WANDB_ENTITY>/<WANDB_PROJECT>/.....